# PyCity Schools Analysis

- Your analysis here
  
---

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from pathlib import Path
from functools import reduce


# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
output_file_path = Path("Resources/school_data_complete.csv")

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.to_csv(output_file_path,index=False)
school_data_complete.head()
#school_data.columns

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools

school_count = len(pd.unique(school_data_complete['school_name']))
school_count
#school_data_complete.columns

15

In [3]:
# Calculate the total number of students
student_count = len(pd.unique(school_data_complete['Student ID']))
student_count

39170

In [4]:
# Calculate the total budget
total_budget = sum(pd.unique(school_data_complete['budget']))
total_budget

24649428

In [5]:
# Calculate the average (mean) math score
average_math_score=school_data_complete['math_score'].mean()
#grouped = school_data_complete.groupby('math_score')
#average_math_score = grouped.mean()
average_math_score 

78.98537145774827

In [6]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean(skipna=False)

average_reading_score

81.87784018381414

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [8]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame

district_summary = pd.DataFrame([{'School Count':int(school_count),'Total Students':int(student_count),'Total Budget':total_budget,'Avg Math Score':average_math_score,'Avg Reading Score':average_reading_score,'Passing Math %':passing_math_percentage,'Passing Reading %':passing_reading_percentage,'Overall Passing Rate':overall_passing_rate}])

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary.style.hide()
#school_data.columns 

School Count,Total Students,Total Budget,Avg Math Score,Avg Reading Score,Passing Math %,Passing Reading %,Overall Passing Rate
15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

In [11]:
# Use the code provided to select all of the school types
school_data1 = school_data
school_types = pd.DataFrame({'school_name':school_data1['school_name'],'Type':school_data1['type']})
school_types 
#school_data.columns 

,school_name,Type
0,Huang High School,District
1,Figueroa High School,District
2,Shelton High School,Charter
3,Hernandez High School,District
4,Griffin High School,Charter
5,Wilson High School,Charter
6,Cabrera High School,Charter
7,Bailey High School,District
8,Holden High School,Charter
9,Pena High School,Charter


In [12]:
# Calculate the total student count per school
per_school_counts = school_data_complete['school_name'].value_counts().reset_index(name='Total Students')
per_school_counts.columns
#school_data.columns 

Index(['school_name', 'Total Students'], dtype='object')

In [13]:
# Calculate the total school budget and per capita spending per school
per_school_budget = school_data.groupby('school_name')['budget'].sum().reset_index(name='Total School Budget')
per_school_capita = school_data_complete.groupby('school_name')['budget'].sum() / school_data_complete.groupby('school_name')['size'].sum()
per_school_budget
per_school_capita = per_school_capita.reset_index(name='Per Student Budget')
#per_school_capita.reset_index(name='Total School Budget')
per_school_capita

,school_name,Per Student Budget
0,Bailey High School,628.0
1,Cabrera High School,582.0
2,Figueroa High School,639.0
3,Ford High School,644.0
4,Griffin High School,625.0
5,Hernandez High School,652.0
6,Holden High School,581.0
7,Huang High School,655.0
8,Johnson High School,650.0
9,Pena High School,609.0


In [14]:
# Calculate the average test scores per school
per_school_math = school_data_complete.groupby('school_name')['math_score'].mean().reset_index(name='Avg Math Score')
per_school_reading = school_data_complete.groupby('school_name')['reading_score'].mean().reset_index(name='Avg Reading Score')
#per_school_math

In [15]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]
school_students_passing_math = students_passing_math.groupby('school_name').size().reset_index(name='Pass_Math')

output_file_path1 = Path("Resources/school_passing_math.csv")
students_passing_math.to_csv(output_file_path1,index=False)
#school_students_passing_math
#students_passing_math

In [16]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]
school_students_passing_reading = students_passing_reading.groupby('school_name').size().reset_index(name ='Pass_Read')


In [17]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]

school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size().reset_index(name='overall')

In [18]:
# Use the provided code to calculate the passing rates
#Math Passing rate calculation. setting the index to  make sure right passing rate for Math/read and overall percentage is calculated from different dataframes
# Initializing.
per_school_passing_math = pd.DataFrame(school_students_passing_math)
per_school_passing_reading = pd.DataFrame(school_students_passing_reading)
per_school_passing_rate1 = pd.DataFrame(school_students_passing_math_and_reading)
#setting index
per_school_passing_math.set_index('school_name',inplace=True)
per_school_passing_reading.set_index('school_name',inplace=True)
per_school_passing_rate1.set_index('school_name',inplace=True)
per_school_counts.set_index('school_name',inplace=True)
#per_school_counts['Total Students'].fillna(0,inplace=True)
#Calculating
per_school_passing_math['%passing Math'] = per_school_passing_math['Pass_Math'] / per_school_counts['Total Students'] * 100
per_school_passing_reading['%passing Read'] = per_school_passing_reading['Pass_Read'] / per_school_counts['Total Students'] * 100
#school_students_passing_reading['Pass_Read'].dtypes
#per_school_passing_math.reset_index(inplace=True)
#per_school_passing_math
#per_school_counts.columns
#per_school_counts['Total Students'].fillna(0,inplace=True)

#per_school_passing_reading.reset_index(inplace=True)
#per_school_passing_reading
per_school_passing_rate1['%Overall Passing'] = per_school_passing_rate1['overall'] / per_school_counts['Total Students'] * 100
#overall_passing_rate
per_school_passing_rate1
#per_school_passing_math_and_reading['overall'].dtype

,overall,%Overall Passing
school_name,,
Bailey High School,2719,54.642283
Cabrera High School,1697,91.334769
Figueroa High School,1569,53.204476
Ford High School,1487,54.289887
Griffin High School,1330,90.599455
Hernandez High School,2481,53.527508
Holden High School,381,89.227166
Huang High School,1561,53.513884
Johnson High School,2549,53.539172


In [19]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary1 = [school_types,per_school_counts,per_school_budget,per_school_capita,per_school_math,per_school_reading,per_school_passing_math,per_school_passing_reading,per_school_passing_rate1]
per_school_summary = reduce(lambda left,right: pd.merge(left,right,on='school_name'),per_school_summary1)
# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary=per_school_summary.sort_values(by='school_name')
# Display the DataFrame
per_school_summary.style.hide()

school_name,Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,Pass_Math,%passing Math,Pass_Read,%passing Read,overall,%Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,3318,66.680064,4077,81.933280,2719,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,1749,94.133477,1803,97.039828,1697,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,1946,65.988471,2381,80.739234,1569,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,1871,68.309602,2172,79.299014,1487,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1371,93.392371,1426,97.138965,1330,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,3094,66.752967,3748,80.862999,2481,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,395,92.505855,411,96.252927,381,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,1916,65.683922,2372,81.316421,1561,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,3145,66.057551,3867,81.222432,2549,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,910,94.594595,923,95.945946,871,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [20]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='%Overall Passing',ascending=False)
top_schools.head(5).style.hide()

school_name,Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,Pass_Math,%passing Math,Pass_Read,%passing Read,overall,%Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,1749,94.133477,1803,97.039828,1697,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,1525,93.272171,1591,97.308869,1487,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1371,93.392371,1426,97.138965,1330,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,2143,93.867718,2204,96.539641,2068,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,910,94.594595,923,95.945946,871,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='%Overall Passing',ascending=True)
bottom_schools.head(5).style.hide()

school_name,Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,Pass_Math,%passing Math,Pass_Read,%passing Read,overall,%Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,2654,66.366592,3208,80.220055,2119,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,1946,65.988471,2381,80.739234,1569,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,1916,65.683922,2372,81.316421,1561,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,3094,66.752967,3748,80.862999,2481,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,3145,66.057551,3867,81.222432,2549,53.539172


## Math Scores by Grade

In [22]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]
#ninth_graders
# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean().reset_index(name='9th Average')
ninth_grade_math_scores
tenth_grader_math_scores = tenth_graders.groupby('school_name')['math_score'].mean().reset_index(name='10th Average')
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean().reset_index(name='11th Average')
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean().reset_index(name='12th Average')

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = [ninth_grade_math_scores,tenth_grader_math_scores,eleventh_grader_math_scores,twelfth_grader_math_scores]
math_scores_by_grade = reduce(lambda left,right: pd.merge(left,right,on='school_name'),math_scores_by_grade)
# Minor data wrangling
#math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade.style.hide()


school_name,9th Average,10th Average,11th Average,12th Average
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [23]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean().reset_index(name='9th')
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean().reset_index(name='10th')
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean().reset_index(name='11th')
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean().reset_index(name='12th')

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = [ninth_grade_reading_scores,tenth_grader_reading_scores,eleventh_grader_reading_scores,twelfth_grader_reading_scores]
reading_scores_by_grade = reduce(lambda left,right: pd.merge(left,right,on='school_name'),reading_scores_by_grade)
# Minor data wrangling
#reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
#reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade.style.hide()

school_name,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [24]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [25]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()
school_spending_df=school_spending_df.rename(columns={'Per Student Budget':'PBudget'})
#school_spending_df

In [26]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df['PBudget']= pd.to_numeric(school_spending_df['PBudget'].replace('[\$,]','',regex=True), errors ='coerce')
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df.PBudget,spending_bins,labels=labels,ordered=False)
school_spending_df.style.hide()

school_name,Type,Total Students,Total School Budget,PBudget,Avg Math Score,Avg Reading Score,Pass_Math,%passing Math,Pass_Read,%passing Read,overall,%Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",628.000000,77.048432,81.033963,3318,66.680064,4077,81.933280,2719,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",582.000000,83.061895,83.975780,1749,94.133477,1803,97.039828,1697,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",639.000000,76.711767,81.158020,1946,65.988471,2381,80.739234,1569,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",644.000000,77.102592,80.746258,1871,68.309602,2172,79.299014,1487,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",625.000000,83.351499,83.816757,1371,93.392371,1426,97.138965,1330,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",652.000000,77.289752,80.934412,3094,66.752967,3748,80.862999,2481,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",581.000000,83.803279,83.814988,395,92.505855,411,96.252927,381,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",655.000000,76.629414,81.182722,1916,65.683922,2372,81.316421,1561,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",650.000000,77.072464,80.966394,3145,66.057551,3867,81.222432,2549,53.539172,$645-680
Pena High School,Charter,962,"$585,858.00",609.000000,83.839917,84.044699,910,94.594595,923,95.945946,871,90.540541,$585-630


In [27]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Avg Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Avg Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["%passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["%passing Read"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["%Overall Passing"].mean()

In [28]:
# Assemble into DataFrame
spending_summary = [spending_math_scores,spending_reading_scores,spending_passing_math,spending_passing_reading,overall_passing_spending]
spending_summary = reduce(lambda left,right: pd.merge(left,right,on='Spending Ranges (Per Student)'),spending_summary)
# Display results
spending_summary

,Avg Math Score,Avg Reading Score,%passing Math,%passing Read,%Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [29]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [30]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_score_summary = per_school_summary.copy()
per_school_score_summary=per_school_score_summary.rename(columns={'Total Students':'tstudents'})
per_school_summary["School Size"] = pd.cut(per_school_score_summary.tstudents,size_bins,labels=labels,ordered=False)
per_school_summary.style.hide()

school_name,Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,Pass_Math,%passing Math,Pass_Read,%passing Read,overall,%Overall Passing,School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,3318,66.680064,4077,81.933280,2719,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,1749,94.133477,1803,97.039828,1697,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,1946,65.988471,2381,80.739234,1569,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,1871,68.309602,2172,79.299014,1487,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1371,93.392371,1426,97.138965,1330,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,3094,66.752967,3748,80.862999,2481,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,395,92.505855,411,96.252927,381,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,1916,65.683922,2372,81.316421,1561,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,3145,66.057551,3867,81.222432,2549,53.539172,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,910,94.594595,923,95.945946,871,90.540541,Small (<1000)


In [31]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Avg Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Avg Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["%passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["%passing Read"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["%Overall Passing"].mean()

In [32]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = [size_math_scores,size_reading_scores,size_passing_math,size_passing_reading,size_overall_passing]
size_summary = reduce(lambda left,right: pd.merge(left,right,on='School Size'),size_summary)
# Display results
size_summary

,Avg Math Score,Avg Reading Score,%passing Math,%passing Read,%Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [33]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["Type"])["Avg Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["Type"])["Avg Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["Type"])["%passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["Type"])["%passing Read"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["Type"])["%Overall Passing"].mean()

In [34]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = [average_math_score_by_type,average_reading_score_by_type,average_percent_passing_math_by_type,average_percent_passing_reading_by_type,average_percent_overall_passing_by_type]
type_summary = reduce(lambda left,right: pd.merge(left,right,on='Type'),type_summary)
# Display results
type_summary

,Avg Math Score,Avg Reading Score,%passing Math,%passing Read,%Overall Passing
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
